<a href="https://colab.research.google.com/github/Gus-oss/DeepLearning/blob/main/Tarea1Parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Librerias importantes
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

#Coneccion con google drive
from google.colab import drive
from google.colab import files
#Importar la carpeta de drive
drive.mount('/content/drive') #coneccion con drive

Mounted at /content/drive


In [3]:
df = pd.read_excel("/content/drive/MyDrive/DeepLearning/archivos/EarlyRetirementPrediction.csv.xlsx")

In [4]:
sc = StandardScaler() # Standardisación
scaled = sc.fit_transform(df)
res = pd.DataFrame(scaled, columns=df.columns) #este es el df que utilizaremos

In [5]:
# Cargar datos (asumiendo que res y df ya existen)
X_columns = ['Gender', 'Desease', 'Education Level', 'Marital Status',
             'Monthly Income', 'Employee/Employer', 'Dependants',
             'Unemployment Rate', 'Stock Market', 'Credit Score',
             'Government Bonds Return']
X_raw = res[X_columns].values
y = df['Retire Before 65 Years Old'].values.reshape(-1, 1)

In [9]:
# Funciones de activación y pérdida
def sigmoid(z):
    z = np.clip(z, -500, 500)
    return 1 / (1 + np.exp(-z))

def sigmoid_prime(z):
    s = sigmoid(z)
    return s * (1 - s)

def cross_entropy_loss(y, y_hat):
    epsilon = 1e-15
    y_hat = np.clip(y_hat, epsilon, 1 - epsilon)
    return -np.sum(y * np.log(y_hat))

# Inicialización de pesos
np.random.seed(42)
n_input = X_raw.shape[1]
n_hidden = 4
n_output = 1

W1 = np.random.randn(n_input, n_hidden) * 0.1
b1 = np.zeros((1, n_hidden))
W2 = np.random.randn(n_hidden, n_output) * 0.1
b2 = np.zeros((1, n_output))

# Forward propagation
def forward(X):
    z1 = X @ W1 + b1
    a1 = sigmoid(z1)
    z2 = a1 @ W2 + b2
    y_hat = sigmoid(z2)
    return z1, a1, z2, y_hat

# Backpropagation (gradiente para -y·log(ŷ))
def backward(X, y, z1, a1, z2, y_hat, eta):
    global W1, b1, W2, b2

    # Gradiente de -y·log(ŷ) respecto a ŷ es: -y/ŷ
    # Multiplicado por sigmoid_prime(z2) para obtener el gradiente
    delta2 = (-y / y_hat) * sigmoid_prime(z2)
    dW2 = a1.T @ delta2
    db2 = np.sum(delta2, axis=0, keepdims=True)

    delta1 = (delta2 @ W2.T) * sigmoid_prime(z1)
    dW1 = X.T @ delta1
    db1 = np.sum(delta1, axis=0, keepdims=True)

    W2 -= eta * dW2
    b2 -= eta * db2
    W1 -= eta * dW1
    b1 -= eta * db1

# Entrenamiento por mini-batches
batch_size = 100
eta = 0.01
N = X_raw.shape[0]
n_batches = N // batch_size

print(f"Mini-batches: {n_batches} de {batch_size} observaciones")

for lote in range(n_batches):
    inicio = lote * batch_size
    fin = inicio + batch_size

    X_batch = X_raw[inicio:fin]
    y_batch = y[inicio:fin]

    z1, a1, z2, y_hat = forward(X_batch)
    loss = cross_entropy_loss(y_batch, y_hat)

    if lote == 0:
        loss_inicial = loss
    if lote == n_batches - 1:
        loss_final = loss

    backward(X_batch, y_batch, z1, a1, z2, y_hat, eta)

    print(f"Lote {lote+1:2d}/{n_batches} | Índices [{inicio:4d}:{fin:4d}] | Loss: {loss:.4f}")



print(f"\n COMPARACIÓN DE COSTOS")
print(f"Loss INICIAL (Lote 1):   {loss_inicial:.6f}")
print(f"Loss FINAL (Lote {n_batches}):    {loss_final:.6f}")
print(f"Diferencia:              {loss_inicial - loss_final:.6f}")
print(f"Reducción:               {(loss_inicial - loss_final) / loss_inicial * 100:.2f}%")

Mini-batches: 15 de 100 observaciones
Lote  1/15 | Índices [   0: 100] | Loss: 35.8361
Lote  2/15 | Índices [ 100: 200] | Loss: 21.0838
Lote  3/15 | Índices [ 200: 300] | Loss: 19.1898
Lote  4/15 | Índices [ 300: 400] | Loss: 13.5643
Lote  5/15 | Índices [ 400: 500] | Loss: 10.2917
Lote  6/15 | Índices [ 500: 600] | Loss: 10.6353
Lote  7/15 | Índices [ 600: 700] | Loss: 7.6247
Lote  8/15 | Índices [ 700: 800] | Loss: 6.8510
Lote  9/15 | Índices [ 800: 900] | Loss: 5.3925
Lote 10/15 | Índices [ 900:1000] | Loss: 4.9793
Lote 11/15 | Índices [1000:1100] | Loss: 5.5135
Lote 12/15 | Índices [1100:1200] | Loss: 4.1728
Lote 13/15 | Índices [1200:1300] | Loss: 4.3209
Lote 14/15 | Índices [1300:1400] | Loss: 3.9363
Lote 15/15 | Índices [1400:1500] | Loss: 3.6523

 COMPARACIÓN DE COSTOS
Loss INICIAL (Lote 1):   35.836122
Loss FINAL (Lote 15):    3.652300
Diferencia:              32.183822
Reducción:               89.81%
